In [1]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

from scipy import stats

In [2]:
train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")
match_df = pd.read_csv("./input/match_reports.csv")

In [3]:
points = match_df[['id','home_team_score','away_team_score']]

In [4]:
points['home_point'] = points['home_team_score']-points['away_team_score']
points['away_point'] = points['away_team_score']-points['home_team_score']

points['home_point'] = points.home_point.map(lambda x: 3 if x>0 else 1 if x==0
                                             else 0)
points['away_point'] = points.away_point.map(lambda x: 3 if x>0 else 1 if x==0
                                             else 0)

In [5]:
points

,id,home_team_score,away_team_score,home_point,away_point
0,9190,1,1,1,1
1,9191,0,2,0,3
2,9192,2,0,3,0
3,9193,1,1,1,1
4,9194,3,2,3,0
5,9195,4,2,3,0
6,9196,6,0,3,0
7,9197,3,4,0,3
8,9198,4,1,3,0
9,9199,3,1,3,0


In [6]:
test_df = test_df.dropna()
test_df.isnull().sum()

id               0
match_date       0
kick_off_time    0
section          0
round            0
home_team        0
away_team        0
venue            0
weather          0
temperature      0
humidity         0
broadcasters     0
dtype: int64

In [7]:
#開催日時をdatetimeに
train_df.match_date = pd.to_datetime(train_df.match_date)
test_df.match_date = pd.to_datetime(test_df.match_date)

In [8]:
# データフレームを一つにまとめる
target = train_df.attendance
total_df = pd.concat([train_df.drop(['attendance'],axis=1), test_df])

In [9]:
total_df = total_df[['id','section','round','match_date','home_team','away_team']]

In [10]:
#年の追加
match_date = total_df.match_date
year_list = []
month_list = []
for i in range(len(match_date)):
    year_list.append(match_date.iloc[i].year)
    month_list.append(match_date.iloc[i].month)
total_df['match_date_year'] = year_list
total_df['match_date_month'] = month_list

In [11]:
total_df.loc[:, "section"] = total_df.loc[:, "section"].apply(lambda x: int(x[1:-1]))
total_df.loc[:, "round"] = total_df.loc[:, "round"].apply(lambda x: int(x[1:-1]))

In [12]:
# 両年とも1シーズンに直した時の18節は7月からなので、7月からのsectionを+17する
sec_index = total_df.query("(match_date_year in [2015, 2016])&(match_date_month >= 7)").index
total_df.loc[sec_index, "section"] = total_df.loc[sec_index, "section"] + 17

In [13]:
total_df

,id,section,round,match_date,home_team,away_team,match_date_year,match_date_month
0,9190,1,1,2006-03-04,G大阪,浦和,2006,3
1,9191,1,2,2006-03-05,甲府,清水,2006,3
2,9192,1,2,2006-03-05,FC東京,大分,2006,3
3,9193,1,2,2006-03-05,磐田,福岡,2006,3
4,9194,1,2,2006-03-05,名古屋,C大阪,2006,3
5,9195,1,2,2006-03-05,大宮,千葉,2006,3
6,9196,1,2,2006-03-05,川崎F,新潟,2006,3
7,9197,1,2,2006-03-05,広島,鹿島,2006,3
8,9198,1,2,2006-03-05,横浜FM,京都,2006,3
9,9199,2,1,2006-03-11,浦和,磐田,2006,3


In [16]:
# 半角と全角が混じっている
teams = set(total_df["home_team"].unique().tolist() + total_df["away_team"].unique().tolist())
print(teams)

{'FC東京', '山形', '仙台', '名古屋', '浦和', '鳥栖', '徳島', '柏', '長崎', 'G大阪', '甲府', '京都', '福岡', '札幌', '横浜FM', '磐田', '広島', '鹿島', '千葉', '清水', '湘南', '大分', 'C大阪', '神戸', '横浜FC', '川崎F', '新潟', '松本', '大宮', '東京V'}


In [15]:
# まとめて置き換える

def make_zen2han_dic(iterable):
    zen = list("ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ" +
               "ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ" +
               "０１２３４５６７８９，、．。（）＿−　")
    han = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789,,..()_- ")
    z2h = {}
    pattern = "|".join(zen)
    for it in iterable:
        if re.search(pattern, it) is None:
            continue
        target = it*1
        for z, h in zip(zen, han):
            target = target.replace(z, h)
        z2h[it] = target
    return z2h

z2h = make_zen2han_dic(teams)
total_df["home_team"].replace(z2h, inplace=True)
total_df["away_team"].replace(z2h, inplace=True)

In [17]:
total_df = pd.merge(total_df,points,on='id')
total_df

,id,section,round,match_date,home_team,away_team,match_date_year,match_date_month,home_team_score,away_team_score,home_point,away_point
0,9190,1,1,2006-03-04,G大阪,浦和,2006,3,1,1,1,1
1,9191,1,2,2006-03-05,甲府,清水,2006,3,0,2,0,3
2,9192,1,2,2006-03-05,FC東京,大分,2006,3,2,0,3,0
3,9193,1,2,2006-03-05,磐田,福岡,2006,3,1,1,1,1
4,9194,1,2,2006-03-05,名古屋,C大阪,2006,3,3,2,3,0
5,9195,1,2,2006-03-05,大宮,千葉,2006,3,4,2,3,0
6,9196,1,2,2006-03-05,川崎F,新潟,2006,3,6,0,3,0
7,9197,1,2,2006-03-05,広島,鹿島,2006,3,3,4,0,3
8,9198,1,2,2006-03-05,横浜FM,京都,2006,3,4,1,3,0
9,9199,2,1,2006-03-11,浦和,磐田,2006,3,3,1,3,0


In [18]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3825 entries, 0 to 3824
Data columns (total 12 columns):
id                  3825 non-null int64
section             3825 non-null int64
round               3825 non-null int64
match_date          3825 non-null datetime64[ns]
home_team           3825 non-null object
away_team           3825 non-null object
match_date_year     3825 non-null int64
match_date_month    3825 non-null int64
home_team_score     3825 non-null int64
away_team_score     3825 non-null int64
home_point          3825 non-null int64
away_point          3825 non-null int64
dtypes: datetime64[ns](1), int64(9), object(2)
memory usage: 388.5+ KB


In [19]:
%%time
#勝ち点の合計を列に追加
home_points=[]
away_points=[]
for i in range(len(total_df)):
    home_points.append(total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                          (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                          (total_df['home_team']==total_df['home_team'].iloc[i])].home_point.sum()+
                        
                        total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                            (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                            (total_df['away_team']==total_df['home_team'].iloc[i])].away_point.sum())
    
    away_points.append(total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                          (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                          (total_df['home_team']==total_df['away_team'].iloc[i])].home_point.sum()+
                        
                        total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                            (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                            (total_df['away_team']==total_df['away_team'].iloc[i])].away_point.sum())

Wall time: 49.2 s


In [20]:
total_df['home_points'] = home_points
total_df['away_points'] = away_points

In [21]:
%%time
#得点の追加
home_scores=[]
away_scores=[]
for i in range(len(total_df)):
    home_scores.append(total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                          (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                          (total_df['home_team']==total_df['home_team'].iloc[i])].home_team_score.sum()+
                        
                        total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                            (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                            (total_df['away_team']==total_df['home_team'].iloc[i])].away_team_score.sum())
    
    away_scores.append(total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                          (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                          (total_df['home_team']==total_df['away_team'].iloc[i])].home_team_score.sum()+
                        
                        total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                            (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                            (total_df['away_team']==total_df['away_team'].iloc[i])].away_team_score.sum())

Wall time: 47.4 s


In [22]:
total_df['home_scores'] = home_scores
total_df['away_scores'] = away_scores

In [23]:
%%time
#失点の追加
home_losses=[]
away_losses=[]
for i in range(len(total_df)):
    home_losses.append(total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                          (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                          (total_df['home_team']==total_df['home_team'].iloc[i])].away_team_score.sum()+
                        
                        total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                            (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                            (total_df['away_team']==total_df['home_team'].iloc[i])].home_team_score.sum())
    
    away_losses.append(total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                          (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                          (total_df['home_team']==total_df['away_team'].iloc[i])].away_team_score.sum()+
                        
                        total_df[(total_df['match_date']<total_df['match_date'].iloc[i]) &
                                            (total_df['match_date_year']==total_df['match_date'].iloc[i].year)&
                                            (total_df['away_team']==total_df['away_team'].iloc[i])].home_team_score.sum())

Wall time: 40.6 s


In [24]:
total_df['home_losses'] = home_losses
total_df['away_losses'] = away_losses

In [25]:
#得失点の追加
total_df['home_goals'] = total_df['home_scores']-total_df['home_losses']
total_df['away_goals'] = total_df['away_scores']-total_df['away_losses']

In [26]:
total_df

,id,section,round,match_date,home_team,away_team,match_date_year,match_date_month,home_team_score,away_team_score,home_point,away_point,home_points,away_points,home_scores,away_scores,home_losses,away_losses,home_goals,away_goals
0,9190,1,1,2006-03-04,G大阪,浦和,2006,3,1,1,1,1,0,0,0,0,0,0,0,0
1,9191,1,2,2006-03-05,甲府,清水,2006,3,0,2,0,3,0,0,0,0,0,0,0,0
2,9192,1,2,2006-03-05,FC東京,大分,2006,3,2,0,3,0,0,0,0,0,0,0,0,0
3,9193,1,2,2006-03-05,磐田,福岡,2006,3,1,1,1,1,0,0,0,0,0,0,0,0
4,9194,1,2,2006-03-05,名古屋,C大阪,2006,3,3,2,3,0,0,0,0,0,0,0,0,0
5,9195,1,2,2006-03-05,大宮,千葉,2006,3,4,2,3,0,0,0,0,0,0,0,0,0
6,9196,1,2,2006-03-05,川崎F,新潟,2006,3,6,0,3,0,0,0,0,0,0,0,0,0
7,9197,1,2,2006-03-05,広島,鹿島,2006,3,3,4,0,3,0,0,0,0,0,0,0,0
8,9198,1,2,2006-03-05,横浜FM,京都,2006,3,4,1,3,0,0,0,0,0,0,0,0,0
9,9199,2,1,2006-03-11,浦和,磐田,2006,3,3,1,3,0,1,1,1,1,1,1,0,0


In [27]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3825 entries, 0 to 3824
Data columns (total 20 columns):
id                  3825 non-null int64
section             3825 non-null int64
round               3825 non-null int64
match_date          3825 non-null datetime64[ns]
home_team           3825 non-null object
away_team           3825 non-null object
match_date_year     3825 non-null int64
match_date_month    3825 non-null int64
home_team_score     3825 non-null int64
away_team_score     3825 non-null int64
home_point          3825 non-null int64
away_point          3825 non-null int64
home_points         3825 non-null int64
away_points         3825 non-null int64
home_scores         3825 non-null int64
away_scores         3825 non-null int64
home_losses         3825 non-null int64
away_losses         3825 non-null int64
home_goals          3825 non-null int64
away_goals          3825 non-null int64
dtypes: datetime64[ns](1), int64(17), object(2)
memory usage: 627.5+ KB


In [28]:
total_df = total_df.ix[:,[0,1,2,3,6,7,4,10,12,14,16,18,5,11,13,15,17,19,8,9]]

total_df = total_df.drop(['home_point','away_point','home_team_score','away_team_score'],axis=1)
total_df

,id,section,round,match_date,match_date_year,match_date_month,home_team,home_points,home_scores,home_losses,home_goals,away_team,away_points,away_scores,away_losses,away_goals
0,9190,1,1,2006-03-04,2006,3,G大阪,0,0,0,0,浦和,0,0,0,0
1,9191,1,2,2006-03-05,2006,3,甲府,0,0,0,0,清水,0,0,0,0
2,9192,1,2,2006-03-05,2006,3,FC東京,0,0,0,0,大分,0,0,0,0
3,9193,1,2,2006-03-05,2006,3,磐田,0,0,0,0,福岡,0,0,0,0
4,9194,1,2,2006-03-05,2006,3,名古屋,0,0,0,0,C大阪,0,0,0,0
5,9195,1,2,2006-03-05,2006,3,大宮,0,0,0,0,千葉,0,0,0,0
6,9196,1,2,2006-03-05,2006,3,川崎F,0,0,0,0,新潟,0,0,0,0
7,9197,1,2,2006-03-05,2006,3,広島,0,0,0,0,鹿島,0,0,0,0
8,9198,1,2,2006-03-05,2006,3,横浜FM,0,0,0,0,京都,0,0,0,0
9,9199,2,1,2006-03-11,2006,3,浦和,1,1,1,0,磐田,1,1,1,0


In [29]:
total_df

,id,section,round,match_date,match_date_year,match_date_month,home_team,home_points,home_scores,home_losses,home_goals,away_team,away_points,away_scores,away_losses,away_goals
0,9190,1,1,2006-03-04,2006,3,G大阪,0,0,0,0,浦和,0,0,0,0
1,9191,1,2,2006-03-05,2006,3,甲府,0,0,0,0,清水,0,0,0,0
2,9192,1,2,2006-03-05,2006,3,FC東京,0,0,0,0,大分,0,0,0,0
3,9193,1,2,2006-03-05,2006,3,磐田,0,0,0,0,福岡,0,0,0,0
4,9194,1,2,2006-03-05,2006,3,名古屋,0,0,0,0,C大阪,0,0,0,0
5,9195,1,2,2006-03-05,2006,3,大宮,0,0,0,0,千葉,0,0,0,0
6,9196,1,2,2006-03-05,2006,3,川崎F,0,0,0,0,新潟,0,0,0,0
7,9197,1,2,2006-03-05,2006,3,広島,0,0,0,0,鹿島,0,0,0,0
8,9198,1,2,2006-03-05,2006,3,横浜FM,0,0,0,0,京都,0,0,0,0
9,9199,2,1,2006-03-11,2006,3,浦和,1,1,1,0,磐田,1,1,1,0


In [30]:
exam = total_df[(total_df['match_date_year']==2018)&(total_df['section']==10)][['match_date_year','section','home_team','home_points','home_goals','home_scores','home_losses']].rename(columns={'home_team':'team','home_points':'points','home_goals':'goals','home_scores':'scores','home_losses':'losses'}).append(
                 total_df[(total_df['match_date_year']==2018)&(total_df['section']==10)][['match_date_year','section','away_team','away_points','away_goals','away_scores','away_losses']].rename(columns={'away_team':'team','away_points':'points','away_goals':'goals','away_scores':'scores','away_losses':'losses'}))
exam

,match_date_year,section,team,points,goals,scores,losses
3753,2018,10,札幌,15,3,12,9
3754,2018,10,鹿島,11,-2,7,9
3755,2018,10,柏,11,0,10,10
3756,2018,10,FC東京,16,3,13,10
3757,2018,10,湘南,9,-3,10,13
3758,2018,10,磐田,14,1,8,7
3759,2018,10,名古屋,7,-9,8,17
3760,2018,10,C大阪,15,1,12,11
3761,2018,10,鳥栖,8,-3,10,13
3753,2018,10,横浜FM,9,-4,11,15


In [31]:
exam = exam.sort_values(by=['points','goals','scores','losses'],ascending=False)

In [32]:
for i in range(len(exam)):
    exam['rank'] = [i for i in range(1,19)]
exam

,match_date_year,section,team,points,goals,scores,losses,rank
3756,2018,10,広島,25,10,12,2,1
3756,2018,10,FC東京,16,3,13,10,2
3761,2018,10,川崎F,15,7,13,6,3
3753,2018,10,札幌,15,3,12,9,4
3760,2018,10,C大阪,15,1,12,11,5
3760,2018,10,仙台,15,0,8,8,6
3754,2018,10,神戸,14,3,15,12,7
3758,2018,10,磐田,14,1,8,7,8
3755,2018,10,浦和,12,0,10,10,9
3755,2018,10,柏,11,0,10,10,10


In [33]:
for year in total_df.match_date_year.unique():
    for sec in total_df[total_df['match_date_year']==year].section.unique():
        exam = total_df[(total_df['match_date_year']==year)&(total_df['section']==sec)][['match_date_year','section','home_team','home_points','home_goals','home_scores','home_losses']].rename(columns={'home_team':'team','home_points':'points','home_goals':'goals','home_scores':'scores','home_losses':'losses'}).append(
                 total_df[(total_df['match_date_year']==year)&(total_df['section']==sec)][['match_date_year','section','away_team','away_points','away_goals','away_scores','away_losses']].rename(columns={'away_team':'team','away_points':'points','away_goals':'goals','away_scores':'scores','away_losses':'losses'}))
        exam = exam.sort_values(by=['points','goals','scores','losses'],ascending=False)
        

In [34]:
ranking=pd.DataFrame()

In [35]:
%%time
for year in total_df.match_date_year.unique():
    for sec in total_df[total_df['match_date_year']==year].section.unique():
        exam = total_df[(total_df['match_date_year']==year)&(total_df['section']==sec)][['match_date_year','section','home_team','home_points','home_goals','home_scores','home_losses']].rename(columns={'home_team':'team','home_points':'points','home_goals':'goals','home_scores':'scores','home_losses':'losses'}).append(
                 total_df[(total_df['match_date_year']==year)&(total_df['section']==sec)][['match_date_year','section','away_team','away_points','away_goals','away_scores','away_losses']].rename(columns={'away_team':'team','away_points':'points','away_goals':'goals','away_scores':'scores','away_losses':'losses'}))
        
        exam = exam.sort_values(by=['points','goals','scores','losses'],ascending=False)
        if sec==1:
            exam['rank'] = [1 for i in range(1,19)]
        else:
            exam['rank'] = [i for i in range(1,19)]

        ranking = ranking.append(exam)

Wall time: 4.11 s


In [36]:
ranking

,match_date_year,section,team,points,goals,scores,losses,rank
0,2006,1,G大阪,0,0,0,0,1
1,2006,1,甲府,0,0,0,0,1
2,2006,1,FC東京,0,0,0,0,1
3,2006,1,磐田,0,0,0,0,1
4,2006,1,名古屋,0,0,0,0,1
5,2006,1,大宮,0,0,0,0,1
6,2006,1,川崎F,0,0,0,0,1
7,2006,1,広島,0,0,0,0,1
8,2006,1,横浜FM,0,0,0,0,1
0,2006,1,浦和,0,0,0,0,1


In [37]:
ranking.to_csv('./input/rank.csv',index=False,encoding='cp932')